# Custom potential

We solve the 1D Gross-Pitaevskii equation with a custom potential.
This is similar to Gross-Pitaevskii equation in 1D example
and we show how to define local potentials attached to atoms, which allows for
instance to compute forces.
The custom potential is actually already defined as `ElementGaussian` in DFTK, and could
be used as is.

In [1]:
using DFTK
using LinearAlgebra

First, we define a new element which represents a nucleus generating
a Gaussian potential.

In [2]:
struct CustomPotential <: DFTK.Element
    α  # Prefactor
    L  # Width of the Gaussian nucleus
end

Some default values

In [3]:
CustomPotential() = CustomPotential(1.0, 0.5);

We extend the two methods providing access to the real and Fourier
representation of the potential to DFTK.

In [4]:
function DFTK.local_potential_real(el::CustomPotential, r::Real)
    -el.α / (√(2π) * el.L) * exp(- (r / el.L)^2 / 2)
end
function DFTK.local_potential_fourier(el::CustomPotential, p::Real)
    # = ∫ V(r) exp(-ix⋅p) dx
    -el.α * exp(- (p * el.L)^2 / 2)
end

!!! tip "Gaussian potentials and DFTK"
    DFTK already implements `CustomPotential` in form of the `DFTK.ElementGaussian`,
    so this explicit re-implementation is only provided for demonstration purposes.

We set up the lattice. For a 1D case we supply two zero lattice vectors

In [5]:
a = 10
lattice = a .* [[1 0 0.]; [0 0 0]; [0 0 0]];

In this example, we want to generate two Gaussian potentials generated by
two "nuclei" localized at positions $x_1$ and $x_2$, that are expressed in
$[0,1)$ in fractional coordinates. $|x_1 - x_2|$ should be different from
$0.5$ to break symmetry and get nonzero forces.

In [6]:
x1 = 0.2
x2 = 0.8
positions = [[x1, 0, 0], [x2, 0, 0]]
gauss     = CustomPotential()
atoms     = [gauss, gauss];

We setup a Gross-Pitaevskii model

In [7]:
C = 1.0
α = 2;
n_electrons = 1  # Increase this for fun
terms = [Kinetic(),
         AtomicLocal(),
         LocalNonlinearity(ρ -> C * ρ^α)]
model = Model(lattice, atoms, positions; n_electrons, terms,
              spin_polarization=:spinless);  # use "spinless electrons"

We discretize using a moderate Ecut and run a SCF algorithm to compute forces
afterwards. As there is no ionic charge associated to `gauss` we have to specify
a starting density and we choose to start from a zero density.

In [8]:
basis = PlaneWaveBasis(model; Ecut=500, kgrid=(1, 1, 1))
ρ = zeros(eltype(basis), basis.fft_size..., 1)
scfres = self_consistent_field(basis; tol=1e-5, ρ)
scfres.energies

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -0.143569468160                   -0.42    8.0    201ms
  2   -0.156034348381       -1.90       -1.10    1.0   54.1ms
  3   -0.156768283137       -3.13       -1.56    2.0   2.13ms
  4   -0.157045707293       -3.56       -2.32    1.0   1.36ms
  5   -0.157055414471       -5.01       -2.96    1.0   1.36ms
  6   -0.157056384110       -6.01       -3.66    1.0   1.26ms
  7   -0.157056402371       -7.74       -3.96    1.0   1.23ms
  8   -0.157056406121       -8.43       -4.34    1.0   1.26ms
  9   -0.157056406896       -9.11       -5.06    1.0   1.24ms


Energy breakdown (in Ha):
    Kinetic             0.0380290 
    AtomicLocal         -0.3163457
    LocalNonlinearity   0.1212603 

    total               -0.157056406896

Computing the forces can then be done as usual:

In [9]:
compute_forces(scfres)

2-element Vector{StaticArraysCore.SVector{3, Float64}}:
 [-0.05567869387153471, -0.0, -0.0]
 [0.05566130054313377, -0.0, -0.0]

Extract the converged total local potential

In [10]:
tot_local_pot = DFTK.total_local_potential(scfres.ham)[:, 1, 1]; # use only dimension 1

Extract other quantities before plotting them

In [11]:
ρ = scfres.ρ[:, 1, 1, 1]        # converged density, first spin component
ψ_fourier = scfres.ψ[1][:, 1]   # first k-point, all G components, first eigenvector

101-element Vector{ComplexF64}:
    -0.5217721671749445 + 0.8208916068103407im
   -0.05362185541501258 + 0.08436277840498078im
    0.06353738627713185 - 0.09995846015313518im
    0.02760043003218438 - 0.043422681656603304im
  -0.004817625845707516 + 0.007578472785840021im
 -0.0073295124625479624 + 0.01153114000161948im
 -0.0010572368712565736 + 0.0016635710203066051im
  0.0011764549555705796 - 0.0018508379948182056im
   0.000506662564510667 - 0.0007971990543494404im
   -7.71402687531831e-5 + 0.00012133629502178972im
                        ⋮
  -7.711294125102854e-5 + 0.00012135505760421947im
  0.0005066969941725069 - 0.0007971179660367583im
  0.0011763674234288087 - 0.0018508096275629852im
 -0.0010574753442348338 + 0.0016634240126732652im
  -0.007329334977072786 + 0.011531186284873545im
  -0.004816705078000398 + 0.007578980916482751im
   0.027600112946275457 - 0.04342299030368265im
    0.06353440632276612 - 0.09996053943669153im
   -0.05362258887784815 + 0.08436232848637008im

Transform the wave function to real space and fix the phase:

In [12]:
ψ = ifft(basis, basis.kpoints[1], ψ_fourier)[:, 1, 1]
ψ /= (ψ[div(end, 2)] / abs(ψ[div(end, 2)]));

using Plots
x = a * vec(first.(DFTK.r_vectors(basis)))
p = plot(x, real.(ψ), label="real(ψ)")
plot!(p, x, imag.(ψ), label="imag(ψ)")
plot!(p, x, ρ, label="ρ")
plot!(p, x, tot_local_pot, label="tot local pot")